<a href="https://colab.research.google.com/github/bofeng2268/qm2/blob/main/random_routes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
!pip install tabulate

In [157]:
import random

def generate_random_coordinates(num_points, central_lat_lon, greater_lat_lon):
    random_coords = []
    while len(random_coords) < num_points:
        lat = random.uniform(greater_lat_lon['min_lat'], greater_lat_lon['max_lat'])
        lon = random.uniform(greater_lat_lon['min_lon'], greater_lat_lon['max_lon'])

        if central_lat_lon['min_lat'] <= lat <= central_lat_lon['max_lat'] and central_lat_lon['min_lon'] <= lon <= central_lat_lon['max_lon']:
            continue
        random_coords.append((lat, lon))

    return random_coords

# Define the boundary coordinates for Central London and Greater London
central_london = {'min_lat': 51.44, 'max_lat': 51.56, 'min_lon': -0.18, 'max_lon': 0.02}
greater_london = {'min_lat': 51.38, 'max_lat': 51.60, 'min_lon': -0.33, 'max_lon': 0.15}

# Generate 30 random coordinates
random_coordinates = generate_random_coordinates(100, central_london, greater_london)

In [158]:
import geopandas as gpd
import plotly.express as px
import pandas as pd

# Load the Greater London shapefile
london_shapefile = gpd.read_file('https://gist.githubusercontent.com/cejast/2cb80a2346b2049ac5d0/raw/c7ea98c7b9204782b6652b29bf3ef3e3b1a187ea/london-topojson.json')

# Extract latitude and longitude from the list of coordinates
latitudes, longitudes = zip(*random_coordinates)

# Create a DataFrame from the coordinates
df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

# Create a scatterplot on a Mapbox map
fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude',
                        hover_name='Latitude', hover_data=['Longitude'],
                        title='Random Coordinates on Greater London Map',
                        mapbox_style="carto-positron",
                        center={"lat": 51.5074, "lon": -0.1278},
                        zoom=10)

# Customize the map as needed
fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
# Show the map
fig.show()


In [159]:
import requests
import json
import pandas as pd
import random
from tabulate import tabulate

def parse_journey_data(journey_data):
    # Check if 'journeys' key is in the response
    if 'journeys' not in journey_data or not journey_data['journeys']:
        return None, None

    shortest_duration = min(journey_data['journeys'], key=lambda x: x['duration'])['duration']
    shortest_journey = next(j for j in journey_data['journeys'] if j['duration'] == shortest_duration)
    transport_changes = len(shortest_journey['legs'])

    return shortest_duration, transport_changes

# DataFrame to store journey data
journey_data_df = pd.DataFrame(columns=['Start Coordinate', 'Disabled Duration', 'Disabled Changes',
                                        'Non-Disabled Duration', 'Non-Disabled Changes'])
# Generate 50 random coordinates

api_key = '97c709273afd48debc44e33e180d3d1c'

for start_point in random_coordinates:

    start_point = f"{start_point[0]}, {start_point[1]}"  #random point
    end_point = '51.517327, -0.059259'  #royal hospital
    url = f'https://api.tfl.gov.uk/Journey/JourneyResults/{start_point}/to/{end_point}'

    # Define headers to include the API key
    headers = {
        'app_key': api_key,
    }

    # Make the request for a disabled person with full step-free access and tube preference
    params_disabled = {
        'accessibilityPreference': 'StepFreeToVehicle,StepFreeToPlatform',
        'mode': 'tube'
    }

    # Make the request for a non-disabled person with tube preference
    params_non_disabled = {
        'mode': 'tube'
    }

    response_disabled = requests.get(url, headers=headers, params=params_disabled)
    journey_disabled = response_disabled.json()

    response_non_disabled = requests.get(url, headers=headers, params=params_non_disabled)
    journey_non_disabled = response_non_disabled.json()

    shortest_duration_disabled, transport_changes_disabled = parse_journey_data(journey_disabled)
    shortest_duration_non_disabled, transport_changes_non_disabled = parse_journey_data(journey_non_disabled)

    new_row = pd.DataFrame([{
    'Start Coordinate': start_point,
    'Disabled Duration': shortest_duration_disabled,
    'Disabled Changes': transport_changes_disabled,
    'Non-Disabled Duration': shortest_duration_non_disabled,
    'Non-Disabled Changes': transport_changes_non_disabled
    }])
    journey_data_df = pd.concat([journey_data_df, new_row], ignore_index=True)

# Replace None values with a specific placeholder (e.g., 'N/A')

journey_data_df = journey_data_df.dropna()
print(tabulate(journey_data_df, headers='keys', tablefmt='pretty'))



+----+------------------------------------------+-------------------+------------------+-----------------------+----------------------+
|    |             Start Coordinate             | Disabled Duration | Disabled Changes | Non-Disabled Duration | Non-Disabled Changes |
+----+------------------------------------------+-------------------+------------------+-----------------------+----------------------+
| 1  | 51.58898760308353, -0.16678005011357064  |        104        |        4         |          56           |          5           |
| 2  | 51.49869066292579, -0.22974098501212067  |        62         |        3         |          59           |          3           |
| 4  | 51.595707298913815, -0.2874715855782733  |        84         |        5         |          64           |          5           |
| 5  |  51.58217751386228, -0.2026307725954527  |        96         |        4         |          68           |          5           |
| 6  |  51.51370487106405, 0.03049199440862338  